In [13]:
!pip install scikit-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.7 MB/s eta 0:00:00
  Created wheel for scikit-bio: filename=scikit_bio-0.6.0-cp310-cp310-linux_x86_64.whl size=2978848 sha256=0bcbc22414b88c4908d1aba693833607abcc46c3e8f7d59250762e6cd89b8a29
  Stored in directory: /root/.cache/pip/wheels/44/54/d7/d48067a8b538ad5e67e28c956204e2e564edd7ae5017d9252e
  Created wheel for hdmedians: filename=hdmedians-0.14.2-cp310-cp310-linux_x86_64.whl size=677340 sha256=75e2c8f2dfd97303ecc093c9c0170131da4543ee57a5da5adb66c01eb8f0191a
  Stored in directory: /root/.cache/pip/wheels/82/8f/0d/0c61130cfad119482ebb95

In [14]:
from scipy.spatial import distance
from scipy.stats import zscore
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import skbio

In [23]:
taxo = pd.read_csv('/content/taxo.csv', index_col=0)
msp = pd.read_csv('/content/vect_atlas.csv', index_col=0)
msptaxo = msp.join(taxo['phylum']).groupby('phylum').sum().T
meta = pd.read_csv('/content/sampleID.csv', index_col=0)

In [24]:
meta = meta.loc[meta.Disease == 'T2D']

In [25]:
meta

,BioProject,type,Age,Gender,BMI,Geography,Sequencer,MgsRichness,GeneRichness,enteroType,Disease,IGC2mappedReadCount,IGC2mappedPercent
sample.ID,,,,,,,,,,,,,
SRS333664,PRJNA422434,Disease,37.0,Male,20.0,China,Illumina Genome Analyzer II,22,61548,ET-Bacteroides,T2D,31125344,83.5
SRS333665,PRJNA422434,Disease,78.0,Male,20.0,China,Illumina Genome Analyzer II,72,264694,ET-Bacteroides,T2D,47994254,85.0
SRS333666,PRJNA422434,Disease,65.0,Female,20.0,China,Illumina Genome Analyzer II,210,545864,ET-Bacteroides,T2D,50265602,80.1
SRS333667,PRJNA422434,Disease,65.0,Female,25.0,China,Illumina Genome Analyzer II,105,323553,ET-Bacteroides,T2D,46070008,85.6
SRS333668,PRJNA422434,Disease,63.0,Male,22.0,China,Illumina Genome Analyzer II,199,577881,ET-Firmicutes,T2D,65377321,83.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERS235635,PRJEB1786,Disease,69.0,Female,22.0,Sweden,Illumina HiSeq 2000,341,625224,ET-Firmicutes,T2D,4536165,79.0
ERS235637,PRJEB1786,Disease,70.0,Female,23.0,Sweden,Illumina HiSeq 2000,288,607031,ET-Firmicutes,T2D,4928260,81.7
ERS235638,PRJEB1786,Disease,70.0,Female,23.0,Sweden,Illumina HiSeq 2000,298,621674,ET-Firmicutes,T2D,7079550,81.0


In [26]:
msptaxo

phylum,Actinobacteria,Bacteroidetes,Candidatus Melainabacteria,Elusimicrobia,Euryarchaeota,Firmicutes,Fusobacteria,Proteobacteria,Spirochaetes,Synergistetes,Tenericutes,Verrucomicrobia,unclassified,unclassified Bacteria,unclassified Eukaryota
DRS087519,3.466843e-06,0.000110,0.000000e+00,0.000000e+00,5.047604e-08,0.000121,1.900431e-07,0.000019,0.000019,0.000000e+00,0.000000e+00,0.000000e+00,3.369490e-05,1.692030e-06,0.0
DRS087520,2.266432e-06,0.000088,0.000000e+00,0.000000e+00,0.000000e+00,0.000158,3.299887e-06,0.000014,0.000000,7.894618e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
DRS087521,4.869256e-06,0.000121,0.000000e+00,0.000000e+00,0.000000e+00,0.000081,1.368882e-08,0.000005,0.000004,0.000000e+00,0.000000e+00,7.105836e-06,4.810709e-06,0.000000e+00,0.0
DRS087522,4.066411e-06,0.000078,0.000000e+00,0.000000e+00,1.718251e-06,0.000196,0.000000e+00,0.000002,0.000000,8.942469e-08,0.000000e+00,0.000000e+00,8.121477e-06,0.000000e+00,0.0
DRS087523,1.430262e-06,0.000176,0.000000e+00,0.000000e+00,0.000000e+00,0.000124,0.000000e+00,0.000005,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.724966e-06,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRS4597849,5.543980e-07,0.000192,4.059764e-07,0.000000e+00,9.175234e-06,0.000097,0.000000e+00,0.000002,0.000000,0.000000e+00,1.347589e-05,4.998091e-08,1.545341e-05,0.000000e+00,0.0
SRS4597848,1.547763e-05,0.000091,8.113542e-09,9.756005e-09,0.000000e+00,0.000065,0.000000e+00,0.000061,0.000000,1.672314e-08,1.332758e-07,4.634563e-08,1.158530e-06,0.000000e+00,0.0
SRS4597847,8.014720e-06,0.000015,0.000000e+00,0.000000e+00,0.000000e+00,0.000096,0.000000e+00,0.000081,0.000000,0.000000e+00,0.000000e+00,1.619385e-07,2.160755e-07,0.000000e+00,0.0
SRS4597846,2.587022e-07,0.000149,1.222917e-07,1.301821e-07,0.000000e+00,0.000107,0.000000e+00,0.000029,0.000000,0.000000e+00,2.058462e-06,2.384661e-07,1.307238e-05,7.336517e-08,0.0


In [28]:
table = meta.join(msptaxo['phylum']).groupby('phylum').sum().T

KeyError: 'phylum'